# Inlämning 2: Numpy, Pandas och Analys

### Part 1: Numpy

In [111]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

Konstruera en array med 10 kolumner och 5 rader av float-tal. Inget tal får vara samma.

In [112]:
first_array = np.random.normal(size=(5, 10))

print(first_array)
print(first_array.dtype)

[[ 0.9119  0.1766  0.2155  1.4058 -0.1256  0.2602  1.3565 -1.4811 -1.0586
  -0.0997]
 [ 1.4217 -0.0547 -0.6164 -0.0919 -0.069   0.195   0.2729  0.5433  0.8753
   0.5941]
 [-0.5193 -0.3488 -1.3786 -0.2957 -0.4615 -0.4502 -0.5109  1.4254  0.2262
   1.7276]
 [-0.3089 -0.5929  2.8169 -0.9216 -0.9419  0.6317 -1.3306  0.2109  3.5793
   1.8114]
 [-1.0048 -0.0395  0.6787 -0.4097  0.3375 -0.161   0.9055  0.7567 -2.664
  -0.0765]]
float64


Konstruera en ny array till med 10 kolumner och 7 rader av float-tal. Den skall inte vara samma som ovan array..

In [113]:
second_array = np.random.randn(7, 10)

print(second_array)
print(second_array.dtype)

[[-0.6712 -2.7182 -0.0529  0.166  -0.1589 -1.5955 -1.2663 -0.015   0.0172
   0.4444]
 [-0.2251 -0.1259 -0.5514 -0.6385  0.5217 -0.5866  1.4209 -0.9672 -0.3507
  -0.506 ]
 [ 0.421  -0.2203  0.0063  0.5081 -0.1357 -0.3168  0.9979 -2.0015 -0.1034
  -0.5977]
 [ 0.0515  0.356   2.001   0.3218  0.7073  0.4841 -0.38   -0.1421  0.0661
  -0.3518]
 [ 1.2861  0.0195 -1.2479  0.6103  0.0254  0.9279 -0.0269 -0.7923 -0.9733
   0.2331]
 [ 1.3926  0.7538 -2.489  -0.5849 -0.3427  0.7563  0.3367 -0.4106  0.7865
   0.4626]
 [-0.3148 -0.0444  1.3677 -1.2522  0.3399 -1.4884  0.9095  0.5344  2.6471
   0.893 ]]
float64


Gör om de ovan arrayerna så att de har 5 kolumner men innehåller samma data.

In [114]:
print(first_array)
print(second_array)
first_array = np.delete(first_array, slice(5, 10), axis=1)
second_array = np.delete(second_array, slice(5, 10), axis=1)
print(first_array)
print(second_array)

[[ 0.9119  0.1766  0.2155  1.4058 -0.1256  0.2602  1.3565 -1.4811 -1.0586
  -0.0997]
 [ 1.4217 -0.0547 -0.6164 -0.0919 -0.069   0.195   0.2729  0.5433  0.8753
   0.5941]
 [-0.5193 -0.3488 -1.3786 -0.2957 -0.4615 -0.4502 -0.5109  1.4254  0.2262
   1.7276]
 [-0.3089 -0.5929  2.8169 -0.9216 -0.9419  0.6317 -1.3306  0.2109  3.5793
   1.8114]
 [-1.0048 -0.0395  0.6787 -0.4097  0.3375 -0.161   0.9055  0.7567 -2.664
  -0.0765]]
[[-0.6712 -2.7182 -0.0529  0.166  -0.1589 -1.5955 -1.2663 -0.015   0.0172
   0.4444]
 [-0.2251 -0.1259 -0.5514 -0.6385  0.5217 -0.5866  1.4209 -0.9672 -0.3507
  -0.506 ]
 [ 0.421  -0.2203  0.0063  0.5081 -0.1357 -0.3168  0.9979 -2.0015 -0.1034
  -0.5977]
 [ 0.0515  0.356   2.001   0.3218  0.7073  0.4841 -0.38   -0.1421  0.0661
  -0.3518]
 [ 1.2861  0.0195 -1.2479  0.6103  0.0254  0.9279 -0.0269 -0.7923 -0.9733
   0.2331]
 [ 1.3926  0.7538 -2.489  -0.5849 -0.3427  0.7563  0.3367 -0.4106  0.7865
   0.4626]
 [-0.3148 -0.0444  1.3677 -1.2522  0.3399 -1.4884  0.9095  0.5344

Slå ihop de bägge arrayerna till en array.

In [115]:
result_array = np.concatenate((first_array, second_array), axis=0)
print(result_array)

[[ 0.9119  0.1766  0.2155  1.4058 -0.1256]
 [ 1.4217 -0.0547 -0.6164 -0.0919 -0.069 ]
 [-0.5193 -0.3488 -1.3786 -0.2957 -0.4615]
 [-0.3089 -0.5929  2.8169 -0.9216 -0.9419]
 [-1.0048 -0.0395  0.6787 -0.4097  0.3375]
 [-0.6712 -2.7182 -0.0529  0.166  -0.1589]
 [-0.2251 -0.1259 -0.5514 -0.6385  0.5217]
 [ 0.421  -0.2203  0.0063  0.5081 -0.1357]
 [ 0.0515  0.356   2.001   0.3218  0.7073]
 [ 1.2861  0.0195 -1.2479  0.6103  0.0254]
 [ 1.3926  0.7538 -2.489  -0.5849 -0.3427]
 [-0.3148 -0.0444  1.3677 -1.2522  0.3399]]


Ändra ordningen på kolumnerna så att de har ordningen kolumn5, kolumn3, kolumn4, kolumn1 och kolumn2.

In [116]:
out = result_array[:, [4, 2, 3, 0, 1]]
print(out)

[[-0.1256  0.2155  1.4058  0.9119  0.1766]
 [-0.069  -0.6164 -0.0919  1.4217 -0.0547]
 [-0.4615 -1.3786 -0.2957 -0.5193 -0.3488]
 [-0.9419  2.8169 -0.9216 -0.3089 -0.5929]
 [ 0.3375  0.6787 -0.4097 -1.0048 -0.0395]
 [-0.1589 -0.0529  0.166  -0.6712 -2.7182]
 [ 0.5217 -0.5514 -0.6385 -0.2251 -0.1259]
 [-0.1357  0.0063  0.5081  0.421  -0.2203]
 [ 0.7073  2.001   0.3218  0.0515  0.356 ]
 [ 0.0254 -1.2479  0.6103  1.2861  0.0195]
 [-0.3427 -2.489  -0.5849  1.3926  0.7538]
 [ 0.3399  1.3677 -1.2522 -0.3148 -0.0444]]


### Part 2: Pandas

In [117]:
import pandas as pd

In [118]:
assults = pd.read_csv('assults.csv')
# print(assults.describe())

##### 1 - Hur många regioner finns med i data?

In [119]:
# print(assults.columns)
unique_region_labels = assults['Region_2013_label'].unique()
# print(unique_region_labels)
print(f'The total number of regions is: {len(unique_region_labels)}')

The total number of regions is: 17


##### 2 - Lista regionerna i bokstavsordning

In [120]:
region_names = unique_region_labels.tolist()
print(sorted(region_names))

['Area Outside Region', 'Auckland Region', 'Bay of Plenty Region', 'Canterbury Region', 'Gisborne Region', "Hawke's Bay Region", 'Manawatu-Wanganui Region', 'Marlborough Region', 'Nelson Region', 'Northland Region', 'Otago Region', 'Southland Region', 'Taranaki Region', 'Tasman Region', 'Waikato Region', 'Wellington Region', 'West Coast Region']


##### 3 - Vad är invånarantalet i varje region

In [121]:
#assults.head(20)
print('Population in mid point 2015:\n')
series_grouped = assults.groupby('Region_2013_label')['Population_mid_point_2015'].sum()
series_grouped

Population in mid point 2015:



Region_2013_label
Area Outside Region             610
Auckland Region             1569890
Bay of Plenty Region         287130
Canterbury Region            586310
Gisborne Region               47370
Hawke's Bay Region           160010
Manawatu-Wanganui Region     234460
Marlborough Region            45340
Nelson Region                 49940
Northland Region             168320
Otago Region                 215030
Southland Region              97190
Taranaki Region              115760
Tasman Region                 49500
Waikato Region               439000
Wellington Region            496890
West Coast Region             32700
Name: Population_mid_point_2015, dtype: int64

##### 4 - Hur många brott begicks i varje region

In [122]:
# assults.head(20)
print('Total number of victims in 2015:\n')
series_grouped = assults.groupby('Region_2013_label')['Victimisations_calendar_year_2015'].sum()
series_grouped

Total number of victims in 2015:



Region_2013_label
Area Outside Region            0
Auckland Region             8010
Bay of Plenty Region        1751
Canterbury Region           2942
Gisborne Region              436
Hawke's Bay Region          1068
Manawatu-Wanganui Region    1270
Marlborough Region           209
Nelson Region                357
Northland Region             905
Otago Region                1142
Southland Region             381
Taranaki Region              493
Tasman Region                161
Waikato Region              2330
Wellington Region           2459
West Coast Region            121
Name: Victimisations_calendar_year_2015, dtype: int64

##### 5 - Skapa en dataframe på de områden med de 10 högsta antalet överfall.

In [123]:
# assults.head()
series_grouped = assults.groupby('Urban_area_2013_label')['Victimisations_calendar_year_2015'].sum()
df_grouped = series_grouped.to_frame(name='Number_of_attacks').reset_index()
df_grouped = df_grouped.sort_values(['Number_of_attacks'], ascending=False).groupby('Urban_area_2013_label').head(10).reset_index()
df_grouped = df_grouped.head(10)
df_grouped


,index,Urban_area_2013_label,Number_of_attacks
0,11,Central Auckland Zone,3286
1,102,Southern Auckland Zone,2770
2,12,Christchurch,2348
3,29,Hamilton Zone,1196
4,98,Rural (Incl.some Off Shore Islands),1056
5,139,Wellington Zone,1019
6,141,Western Auckland Zone,900
7,110,Tauranga,693
8,18,Dunedin,657
9,69,Northern Auckland Zone,635


##### 6 - Lägg till en kolumn i dataframen som visar % antal överfall i området jämfört med invånarantal i området

In [124]:
#assults.head()
def condition(x):
    if x['Victimisations_calendar_year_2015'] != 0 and x['Population_mid_point_2015'] !=0:
        return x['Victimisations_calendar_year_2015'] / x['Population_mid_point_2015'] * 100   
    else:
        return 0

assults_copy = assults
assults_copy['Procent_attacks_per_population'] = assults_copy.apply(condition, axis=1)
assults_copy.head()

,Index,Area_unit_2013_code,Area_unit_2013_label,Victimisations_calendar_year_2015,Population_mid_point_2015,Rate_per_10000_population,Rate_ratio_NZ_average_rate,Urban_area_2013_code,Urban_area_2013_label,Urban_area_type,Territorial_authority_area_2013_code,Territorial_authority_area_2013_label,Region_2013_code,Region_2013_label,Procent_attacks_per_population
0,1,514102,Auckland Central West,550,15900,346,6.6,4,Central Auckland Zone,Main urban area,76,Auckland,2,Auckland Region,3.459119
1,2,573101,Willis Street-Cambridge Terrace,447,8300,539,10.3,20,Wellington Zone,Main urban area,47,Wellington City,9,Wellington Region,5.385542
2,3,529700,Hamilton Central,380,3170,"1,199",22.9,6,Hamilton Zone,Main urban area,16,Hamilton City,3,Waikato Region,11.987382
3,4,514103,Auckland Central East,375,13250,283,5.4,4,Central Auckland Zone,Main urban area,76,Auckland,2,Auckland Region,2.830189
4,5,591500,Cathedral Square,305,570,"5,351",102.3,22,Christchurch,Main urban area,60,Christchurch City,13,Canterbury Region,53.508772


##### 7 - Skapa en ny dataframe som innehåller antalet områden som finns av de olika typerna av urban area. Ledtråd: groupby

In [125]:
# assults.head()
series_grouped = assults.groupby('Urban_area_type').size()
df_grouped = series_grouped.to_frame(name='Number_of_areas').reset_index()
print(df_grouped)

        Urban_area_type  Number_of_areas
0       Main urban area             1112
1      Minor urban area              137
2            Rural area              640
3  Secondary urban area              131


##### 8 - Ta reda på hur många områden hade population men inga överfall. Jämfört totala populationen i dessa med totala poplulationen i deras region.

In [132]:
assults_copy = assults
assults_copy = assults_copy[assults_copy['Population_mid_point_2015'] != 0]
assults_copy = assults_copy[assults_copy['Victimisations_calendar_year_2015'] != 0].reset_index()
assults_copy

##### 9 - Räkna antalet områden som inte har någon data på överfall eller invånare.

##### 10 - VG: Skapa en ny dataframe med 10 områderna med mer än 5000 invånare, men har de lägsta överfallen per invånare.

##### 11 - VG: Skapa en ny dataframe som innehåller % överfall i regionen jämfört med antalet invånare i regionen.

##### 12 - VG: Lägg till en ny kolumn i dataframen som visar % antal överfall per region

### Part 3: Utforskning/analys

Minst 500 rader, 5 Kolumner i ditt valda dataset

#### Step 1: Välj dataset

Hitta ett intressant dataset. Det valda datasettet måste vara i CSV format,  ha minst 500 rader och 5 kolumner. 
Ladda ner datasetet med pandas read_csv funktion och en url (se nedan). Alternativt i inlämningen skicka med datasettet.

In [ ]:
url = 'https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv'
data = pd.read_csv(url)

#### Step 2: Data preparation och Cleaning

Ladda in dataset in i en dataframe med pandas. 
Behandla missade data, icke-korrekt data och fel data.
Gör andra steg som behövs (gör om sträng datum till riktiga datum, skapa fler kolumner, slå ihop datasets osv).
Summera hur datasetet ser ut i nuvarande läge i dina markdowns. Storlek, kolumner, kategori typer (qualitative vs. quantitative), kvalitet på data, fång osv.

#### Step 3: Undersökande analys och visualisering

Undersök data genom analysera (mean, sum, range osv)  intressant statistik i numeriska kolumnerna.
Undersök distributions av numeriska kolumner genom histogram osv
Undersök relationen mellan kolumner med scatter plots, bar chars osv.
Notera intressanta upptäckter i din markdown.

#### Step 4: För VG - fråga och svara på 4 intressanta frågor om datan

#### Step 5: Summera dina slutsatser & Skriv ett avslut.